In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import plotly.plotly as py
import plotly.tools as tls

import re

In [2]:
# Загружаем файл routes_by_stop
rbs_df = pd.read_csv('rbs.csv', ';')
rbs_df[rbs_df['ROUTE_NUMBER'] == '467']

,ID_ROUTE,ID_ROUTE_ASUGPT,ID_TRANSPORT,ROUTE_NUMBER,ROUTE_NAME,STOP_NUMBER,DIRECTION,ID_STOP,ID_NEXT_STOP,DISTANCE_GPS,DISTANCE,DDATE
14142,15599,1992,1,467,"СТ. МЕТРО ""УЛ. ДЫБЕНКО"" - 53 КМ АВТОМОБИЛЬНАЯ ...",0,2,2530,4398,"17,32","17,58",03.10.2018
14143,15599,1992,1,467,"СТ. МЕТРО ""УЛ. ДЫБЕНКО"" - 53 КМ АВТОМОБИЛЬНАЯ ...",1,2,4398,30681,"12,55",13,03.10.2018
14144,15599,1992,1,467,"СТ. МЕТРО ""УЛ. ДЫБЕНКО"" - 53 КМ АВТОМОБИЛЬНАЯ ...",2,2,30681,3006,"11,47","12,5",03.10.2018
14145,15599,1992,1,467,"СТ. МЕТРО ""УЛ. ДЫБЕНКО"" - 53 КМ АВТОМОБИЛЬНАЯ ...",3,2,3006,4403,"2,55","2,6",03.10.2018
14146,15599,1992,1,467,"СТ. МЕТРО ""УЛ. ДЫБЕНКО"" - 53 КМ АВТОМОБИЛЬНАЯ ...",4,2,4403,18506,"0,71","0,9",03.10.2018
14147,15599,1992,1,467,"СТ. МЕТРО ""УЛ. ДЫБЕНКО"" - 53 КМ АВТОМОБИЛЬНАЯ ...",0,1,18506,22763,"0,84","0,9",03.10.2018
14148,15599,1992,1,467,"СТ. МЕТРО ""УЛ. ДЫБЕНКО"" - 53 КМ АВТОМОБИЛЬНАЯ ...",1,1,22763,30680,"14,22",15,03.10.2018
14149,15599,1992,1,467,"СТ. МЕТРО ""УЛ. ДЫБЕНКО"" - 53 КМ АВТОМОБИЛЬНАЯ ...",2,1,30680,4379,"12,53","13,3",03.10.2018
14150,15599,1992,1,467,"СТ. МЕТРО ""УЛ. ДЫБЕНКО"" - 53 КМ АВТОМОБИЛЬНАЯ ...",3,1,4379,20595,"17,4",17,03.10.2018
45607,15599,1992,1,467,"СТ. МЕТРО ""УЛ. ДЫБЕНКО"" - 53 КМ АВТОМОБИЛЬНАЯ ...",0,2,2530,4398,"17,32","17,58",01.10.2018


In [3]:
# Вспомогательная функция, которая по номеру маршрута, дате и типу транспорта выдает список остановок в указанную дату в двух направлениях
# Формат результата (4 параметра): количество остановок в прямом направлении, перечень остановок в прямом направлении, 
# количество остановок в обратном направлении, перечень остановок в обратном направлении

def get_route_stops(route_number, tr_date, tr_id):
    route_df = rbs_df[rbs_df['ROUTE_NUMBER'] == route_number]
    route_df = route_df[route_df['DDATE'] == tr_date]
    route_df = route_df[route_df['ID_TRANSPORT'] == tr_id]
    route_stops_in_dir1 = route_df[route_df['DIRECTION'] == 1].ID_STOP.values
    route_stops_in_dir2 = route_df[route_df['DIRECTION'] == 2].ID_STOP.values
    num1 = route_stops_in_dir1.shape[0]
    num2 = route_stops_in_dir2.shape[0]
    if route_stops_in_dir1.shape[0] != 0:
        dir1_str = str(route_stops_in_dir1[0])
    else: 
        dir1_str = ""
    if route_stops_in_dir2.shape[0] != 0:
        dir2_str = str(route_stops_in_dir2[0])
    else: 
        dir2_str = ""
    for stop in route_stops_in_dir1[1:]:
        dir1_str += ","
        dir1_str += str(stop)
    for stop in route_stops_in_dir2[1:]:
        dir2_str += ","
        dir2_str += str(stop)
    return num1, dir1_str, num2, dir2_str

In [4]:
# Пример работы функции get_route_stops
route_number = '68'
tr_date = '03.10.2018'
tr_id = 1 # это значит, что мы хотим получить информацию об автобусе №68

num1, ex1, num2, ex2 = get_route_stops(route_number, tr_date, tr_id)

print('Количество остановок на городском маршруте №' + route_number + ' в день ' + tr_date + ' в направлении 1: ' + str(num1))
print('Количество остановок на городском маршруте №' + route_number + ' в день ' + tr_date + ' в направлении 2: ' + str(num2))

Количество остановок на городском маршруте №68 в день 03.10.2018 в направлении 1: 26
Количество остановок на городском маршруте №68 в день 03.10.2018 в направлении 2: 27


In [58]:
#Исследуем файл rbs на предмет поиска дат, в которые было больше всего информации о маршрутах
max_val = 0
days = []
months = []

for i in ['01', '02','03','04','05','06','07','08','09', '10',
         '11', '12','13','14','15','16','17','18','19', '20',
         '21', '22','23','24','25','26','27','28','29', '30', '31']:
    for j in ['08', '09', '10']:
        cur_date = i + '.' + j + '.2018'
        num_routes = rbs_df[rbs_df['DDATE'] == cur_date]['ROUTE_NUMBER'].unique().shape[0] 
        if num_routes > max_val:
            max_val = num_routes
            
for i in ['01', '02','03','04','05','06','07','08','09', '10',
         '11', '12','13','14','15','16','17','18','19', '20',
         '21', '22','23','24','25','26','27','28','29', '30', '31']:
    for j in ['08', '09', '10']:
        cur_date = i + '.' + j + '.2018'
        num_routes = rbs_df[rbs_df['DDATE'] == cur_date]['ROUTE_NUMBER'].unique().shape[0] 
        if num_routes == max_val:
            days.append(cur_date.split('.')[0])
            months.append(cur_date.split('.')[1])
            
print("Информация о самом большом количестве маршрутов (" + str(max_val) + " маршрута) отражена в документе в следующие дни:")
for day, month in zip(days, months):
    print(day + '.' + month + '.2018')

Информация о самом большом количестве маршрутов (662 маршрута) отражена в документе в следующие дни:
06.09.2018
07.09.2018
08.09.2018
13.09.2018
14.09.2018
18.09.2018
19.09.2018
20.09.2018
21.09.2018
22.09.2018


In [59]:
# Функция, которая позволяет создать единый csv-файл, в котором будет отражена 
# вся информация о маршрутах определенного типа (автобусы в т.ч. коммерческие, трамваи, троллейбусы) в указанный день 
# и остановках на них (в прямом и обратном направлении)

def create_routes_by_stops_csv(tr_date, tr_id):
    all_routes_df = rbs_df[rbs_df['DDATE'] == tr_date]
    all_routes = all_routes_df[all_routes_df['ID_TRANSPORT'] == tr_id]['ROUTE_NUMBER'].unique()
    result_data = []
    for i in range(all_routes.shape[0]):
        route_number = all_routes[i]
        dir1 = ""
        dir2 = ""
        num1, dir1, num2, dir2 = get_route_stops(route_number, tr_date, tr_id)
        result_data.append([str(route_number), tr_id, num1, dir1, num2, dir2])
    return result_data

In [5]:
# Итоговый csv будет содержать 6 колонок: 
# route_number - номер маршрута, 
# id_transport - id вида транспорта, к которому относится маршрут (1 - автобус, в т.ч. коммерческие; 2 - трамвай; 3 - троллейбус),
# number_of_stops_in_direction1 - количество остановок в прямом направлении, 
# stops_in_direction1 - перечень ID остановок в прямом направлении через запятую в строковом формате, 
# number_of_stops_in_direction2 - количество остановок в обратном направлении, 
# stops_in_direction2 - перечень ID остановок в обратном направлении через запятую в строковом формате

# Выбираем дату (по умолчанию стоит дата, в которую больше всего записей о маршрутах)
tr_date = '22.09.2018'

# Готовим даннные для всех типов транспорта
bus_data = create_routes_by_stops_csv(tr_date, 1)
tram_data = create_routes_by_stops_csv(tr_date, 2)
troll_data = create_routes_by_stops_csv(tr_date, 3)

# Записываем итоговые dataset-ы в файлы (3 отдельных csv файла для автобусов, трамваев и троллебусов)
routes_by_stops_df = pd.DataFrame(np.array(bus_data), columns=['route_number', 'id_transport', 'number_of_stops_in_direction1', 'stops_in_direction1', 'number_of_stops_in_direction2', 'stops_in_direction2'])
routes_by_stops_df.to_csv('buses_routes_by_stops_' + tr_date.split('.')[0] + '_' + tr_date.split('.')[1] + '_' + tr_date.split('.')[2] + '.csv', encoding='utf-8-sig')

routes_by_stops_df = pd.DataFrame(np.array(tram_data), columns=['route_number', 'id_transport', 'number_of_stops_in_direction1', 'stops_in_direction1', 'number_of_stops_in_direction2', 'stops_in_direction2'])
routes_by_stops_df.to_csv('tram_routes_by_stops_' + tr_date.split('.')[0] + '_' + tr_date.split('.')[1] + '_' + tr_date.split('.')[2] + '.csv', encoding='utf-8-sig')

routes_by_stops_df = pd.DataFrame(np.array(troll_data), columns=['route_number', 'id_transport', 'number_of_stops_in_direction1', 'stops_in_direction1', 'number_of_stops_in_direction2', 'stops_in_direction2'])
routes_by_stops_df.to_csv('troll_routes_by_stops_' + tr_date.split('.')[0] + '_' + tr_date.split('.')[1] + '_' + tr_date.split('.')[2] + '.csv', encoding='utf-8-sig')


NameError: name 'create_routes_by_stops_csv' is not defined

In [86]:
# Открываем получившися dataset
bus_df = pd.read_csv('buses_routes_by_stops_22_09_2018.csv', index_col=0)
tram_df = pd.read_csv('tram_routes_by_stops_22_09_2018.csv', index_col=0)
troll_df = pd.read_csv('troll_routes_by_stops_22_09_2018.csv', index_col=0)

# Для удобства также создадим общий dataset, в котором будет содержаться информация о всех видах транспорта
r_df = bus_df
r_df = r_df.append(tram_df, ignore_index = True)
r_df = r_df.append(troll_df, ignore_index = True)
r_df = r_df.fillna('-1')

# Блок корректировки данных о маршрутах (дальнешая проверка выявила ошибки в исходных данных об остановках)
r_df.loc[r_df['route_number'] == '467', 'number_of_stops_in_direction2'] = 4
r_df.loc[r_df['route_number'] == '467', 'stops_in_direction2'] = '2530,4398,3006,4403'
r_df.loc[r_df['route_number'] == '467', 'number_of_stops_in_direction1'] = 3
r_df.loc[r_df['route_number'] == '467', 'stops_in_direction1'] = '18506,22763,4379'

r_df.loc[r_df['route_number'] == '469', 'number_of_stops_in_direction1'] = 13
r_df.loc[r_df['route_number'] == '469', 'stops_in_direction1'] = '18506,22763,4379,1310,2026,1313,1314,2529,20595,17650,17652,1319,1320'
r_df.loc[r_df['route_number'] == '469', 'number_of_stops_in_direction2'] = 14
r_df.loc[r_df['route_number'] == '469', 'stops_in_direction2'] = '22764,20596,2533,2532,17654,2530,22002,2528,2527,3212,2524,4398,3006,4403'

In [62]:
# Ищем нужные нам коммерческие маршруты (проверка актуальности информации)
# Данные 11 коммерческих маршрутов - самые популярные среди пассажиров, оплачивающих проезд картами "Подорожник"
# Именно поэтому они представляют наибольший интерес

route_nums = ['К-68', 'К-191', 'К-92', 'К-49', 'К-55', 'К-13', 'К-7', 'К-67', 'К-184', 'К-1', 'К-226']

print("Результаты поиска данных о самых популярных коммерческих маршрутах в документе rbs.csv")
print()

bus_df = r_df[r_df['id_transport'] == 1]

for route_num in route_nums:
    print('----------------------------------------')
    print('Маршрут ', route_num)
    print('Количество остановок в прямом направлении: ', bus_df[bus_df['route_number'] == route_num]['number_of_stops_in_direction1'].values[0])
    print('Количество остановок в обратном направлении: ', bus_df[bus_df['route_number'] == route_num]['number_of_stops_in_direction2'].values[0])

Результаты поиска данных о самых популярных коммерческих маршрутах в документе rbs.csv

----------------------------------------
Маршрут  К-68
Количество остановок в прямом направлении:  3
Количество остановок в обратном направлении:  3
----------------------------------------
Маршрут  К-191
Количество остановок в прямом направлении:  7
Количество остановок в обратном направлении:  7
----------------------------------------
Маршрут  К-92
Количество остановок в прямом направлении:  2
Количество остановок в обратном направлении:  2
----------------------------------------
Маршрут  К-49
Количество остановок в прямом направлении:  6
Количество остановок в обратном направлении:  5
----------------------------------------
Маршрут  К-55
Количество остановок в прямом направлении:  2
Количество остановок в обратном направлении:  2
----------------------------------------
Маршрут  К-13
Количество остановок в прямом направлении:  7
Количество остановок в обратном направлении:  5
-----------------

Как мы видим, информация по остановкам для нужных нам коммерческих маршрутов (тех, по которым есть много информации о транзакциях по картам "Подорожник") некорректна (т.к. фактически на указанных маршрутах гораздо больше остановок).

Для корректной дальнешей работы был проведен дополнительный анализ указанных маршрутов и корректная информация о них описана ниже. Тем не менее, отдельный интерес представляют также те коммерческие маршруты, для которых информация по количеству остановок корректна. Поэтому следующий блок содержит код функции, позволяющей находить коммерческие маршруты, для которых описанное количество остановок дотаточно большое.

In [63]:
# Вначале определим полный список коммерческих маршрутов, информация о которых есть в документе
commercial_routes = []
for route_num in r_df['route_number'].values:
    if str(route_num).split('-')[0] == 'К':
        commercial_routes.append(route_num)

# Далее создадим функцию, позволяющую определить топ 10 коммерческих маршрутов с самым большим количеством остановок
def find_most_informative_routes():
    number_of_stops_1 = []
    number_of_stops_2 = []
    for num in commercial_routes:
        number_of_stops_1.append(r_df[r_df['route_number'] == num]['number_of_stops_in_direction1'].values[0])
        number_of_stops_2.append(r_df[r_df['route_number'] == num]['number_of_stops_in_direction2'].values[0])
    zipped_info = zip(commercial_routes, number_of_stops_1, number_of_stops_2)
    new_zip = sorted(zipped_info, key = lambda x: x[1], reverse=True)
    return new_zip[:10]

In [64]:
# Самые информативные маршруты
res = find_most_informative_routes()
res

[('К-339', 20, 22),
 ('К-226', 19, 21),
 ('К-175', 18, 16),
 ('К-35', 18, 15),
 ('К-10', 17, 18),
 ('К-369', 17, 14),
 ('К-403', 13, 11),
 ('К-43', 13, 13),
 ('К-45', 13, 14),
 ('К-344', 13, 12)]

In [65]:
# Проверка информации по похожим маршрутам городского транспорта
bus_route_nums = ['68', '103', '153', '27', '92', '168', '114', '9', '127', '172', '171', '134А','13', '13А', '39', '187', '49', '181','71', '66', '35', '2', '6', '71', '26']
troll_route_nums = [37, 38, 27, 35]

bus_df = r_df[r_df['id_transport'] == 1]
#tram_df = r_df[r_df['id_transport'] == 2]
troll_df = r_df[r_df['id_transport'] == 3]

print("Результаты поиска данных о нужных автобусах в документе rbs.csv")
print()
for bus_route_num in bus_route_nums:
    print('----------------------------------------')
    print('Маршрут ', bus_route_num)
    print('Количество остановок в прямом направлении: ', bus_df[bus_df['route_number'] == bus_route_num]['number_of_stops_in_direction1'].values[0])
    print('Количество остановок в обратном направлении: ', bus_df[bus_df['route_number'] == bus_route_num]['number_of_stops_in_direction2'].values[0])

print()
print("Результаты поиска данных о нужных троллейбусах в документе rbs.csv")
print()
for troll_route_num in troll_route_nums:
    print('----------------------------------------')
    print('Маршрут ', troll_route_num)
    print('Количество остановок в прямом направлении: ', troll_df[troll_df['route_number'] == troll_route_num]['number_of_stops_in_direction1'].values[0])
    print('Количество остановок в обратном направлении: ', troll_df[troll_df['route_number'] == troll_route_num]['number_of_stops_in_direction2'].values[0])

Результаты поиска данных о нужных автобусах в документе rbs.csv

----------------------------------------
Маршрут  68
Количество остановок в прямом направлении:  26
Количество остановок в обратном направлении:  27
----------------------------------------
Маршрут  103
Количество остановок в прямом направлении:  37
Количество остановок в обратном направлении:  43
----------------------------------------
Маршрут  153
Количество остановок в прямом направлении:  36
Количество остановок в обратном направлении:  46
----------------------------------------
Маршрут  27
Количество остановок в прямом направлении:  30
Количество остановок в обратном направлении:  31
----------------------------------------
Маршрут  92
Количество остановок в прямом направлении:  15
Количество остановок в обратном направлении:  14
----------------------------------------
Маршрут  168
Количество остановок в прямом направлении:  11
Количество остановок в обратном направлении:  12
--------------------------------------

In [8]:
# DEBUG! Убрать 
cur = r_df[r_df['route_number'] == '27']
arr = cur['stops_in_direction2'].values[0]
arr

'3102, 3103, 1676, 2694, 2770, 4302, 3271, 2063, 1476, 2864, 3278, 3308, 2992, 2774, 2089, 3536, 2083, 1925, 1538, 4195, 1847, 4115, 3485, 4308, 3116, 22105, 3114, 3130, 2988, 3032, 1651'

In [7]:
# Следующий блок кода посвящен добавлению в получившийся датасет с информацией об общественном транспорте информацию о кластерах
# (districts), которым принадлежат остановки на маршруте

stations_df = pd.read_csv('stations.csv', ';')
clusters_df = pd.read_csv('district_stops.csv', ';')

def find_cluster_id(id_stop):
    res = clusters_df[clusters_df['ID_STOP'] == int(float(id_stop))]['ID_STOP_GROUP'].values
    if res.shape[0] == 0:
        return '-1'
    else:
        return str(int(res[0]))


In [129]:
stops_in_dir1 = r_df['stops_in_direction1'].values
clusters_column_data1 = []

for stops_list in stops_in_dir1:
    if stops_list == '-1':
        clusters_column_data1.append('-1')
    else:
        stops = stops_list.split(',')
        clusters_str = find_cluster_id(int(stops[0]))
        for stop in stops[1:]:
            clusters_str = clusters_str + ',' + find_cluster_id(stop)
        clusters_column_data1.append(clusters_str)
    
stops_in_dir2 = r_df['stops_in_direction2'].values
clusters_column_data2 = []
for stops_list in stops_in_dir2:
    if stops_list == '-1':
        clusters_column_data2.append('-1')
    else:
        stops = stops_list.split(',')
        clusters_str = find_cluster_id(int(stops[0]))
        for stop in stops[1:]:
            clusters_str = clusters_str + ',' + find_cluster_id(int(stop))
        clusters_column_data2.append(clusters_str)

        
r_df['clusters_in_direction1'] = None
r_df['clusters_in_direction2'] = None
r_df.loc[:,'clusters_in_direction1'] = pd.Series(np.array(clusters_column_data1))
r_df.loc[:,'clusters_in_direction2'] = pd.Series(np.array(clusters_column_data2)) 

r_df.to_csv('all_routes_info.csv', encoding='utf-8-sig')

In [21]:
r_df = pd.read_csv('all_routes_info.csv', index_col=0)
r_df = r_df[['route_number', 'id_transport', 'number_of_stops_in_direction1', 'stops_in_direction1', 'number_of_stops_in_direction2', 'stops_in_direction2', 'clusters_in_direction1', 'clusters_in_direction2']]
r_df[r_df['route_number'] == '38']

,route_number,id_transport,number_of_stops_in_direction1,stops_in_direction1,number_of_stops_in_direction2,stops_in_direction2,clusters_in_direction1,clusters_in_direction2
214,38,1,20,"23273,3681,20618,2641,1548,1728,2699,22485,273...",21,"30516,19908,19906,19904,19902,19900,19897,1989...","202,202,2480,469,903,1345,718,966,797,797,859,...","1863,753,885,816,2398,978,1186,2531,321,1520,8..."
689,38,2,18,"16404,16430,16436,16455,16461,15965,15967,1597...",19,"17000,16702,16524,16528,16567,16571,16550,1614...","198,198,1495,2185,203,203,1741,1743,1821,1216,...","839,839,1354,74,74,1232,2086,1822,2336,1742,12..."
724,38,3,35,"16959,16960,16958,15540,15562,15571,15572,1557...",31,"15832,14946,15591,14954,15583,15578,16780,1496...","1941,1941,2059,602,605,1220,604,603,600,600,19...","417,2325,1364,294,860,2179,1020,1498,1018,1723..."


In [11]:
stops_191_2 = '21667,3463,26825,1351,1531,2186,2223,1788,4376,3069,1664,1535,4215,2744,2730,4314,3374,3504,3086,3414,1892,17839,2058,2677,2010,1462,2377,7069,3615,3096,4589,2285,22481,20777'
clusters_191_2 = find_cluster_id(stops_191_2.split(',')[0])
for stop in stops_191_2.split(',')[1:]:
    clusters_191_2 = clusters_191_2 + ',' + find_cluster_id(stop)
clusters_191_2

'416,2325,1364,294,860,2179,1020,1498,1018,876,875,877,361,878,198,1495,2185,203,203,1741,1743,1821,1216,606,600,603,1716,1712,2529,1752,1940,2338,2210,2058'

In [12]:
stops_191_1 = "20777,3391,3471,3711,3713,7070,4603,3717,2675,3197,1465,3247,1890,2352,3085,2232,1893,2321,3718,1780,1522,1536,4206,2743,2869,1880,4374,4375,2976,3406,3373,1768,2739,15589,26825,26826,21667"
clusters_191_1 = find_cluster_id(stops_191_1.split(',')[0])
for stop in stops_191_1.split(',')[1:]:
    clusters_191_1 = clusters_191_1 + ',' + find_cluster_id(stop)
clusters_191_1

'2058,2210,2338,1940,1752,2529,1712,1716,603,600,606,1216,1740,1821,1743,1741,203,203,2185,1491,198,878,361,877,875,876,876,1018,1498,1020,2179,1455,860,294,1364,2325,416'

In [15]:
comm_df = pd.read_csv('commerce_stops.csv', index_col=0)

In [16]:
comm_df

,route_number,number_of_stops_in_direction1,stops_in_direction1,number_of_stops_in_direction2,stops_in_direction2,clusters_in_direction1,clusters_in_direction2
0,191,37,"20777,3391,3471,3711,3713,7070,4603,3717,2675,...",34,"21667,3463,26825,1351,1531,2186,2223,1788,4376...","2058,2210,2338,1940,1752,2529,1712,1716,603,60...","416,2325,1364,294,860,2179,1020,1498,1018,876,..."
